In [51]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas_ta as ta
import plotly.io as pio

pio.renderers.default = "browser"

In [52]:
ticker = 'SPY'
start_date = '2018-01-01'
end_date = '2025-09-22'
data = yf.download(ticker, start=start_date, end=end_date, auto_adjust=True)

if isinstance(data.columns, pd.MultiIndex):
    data.columns = data.columns.droplevel(1)
data.columns = data.columns.str.lower()

[*********************100%***********************]  1 of 1 completed


In [53]:
data.ta.mcgd(length=14, c=1.618, append=True)
data.ta.psar(append=True)
data.columns = data.columns.str.lower()
data.rename(columns={'mcgd_14': 'mcginley'}, inplace=True)
data['parabolic_sar'] = data['psarl_0.02_0.2'].fillna(data['psars_0.02_0.2'])

In [54]:
data.dropna(subset=['mcginley', 'parabolic_sar'], inplace=True)

data['regime'] = np.where(data['close'] > data['mcginley'], 1, -1)
sar_position = pd.Series(np.where(data['close'] > data['parabolic_sar'], 1, -1), index=data.index)
data['sar_flip'] = sar_position.diff()
data['signal'] = 0
data.loc[(data['regime'] == 1) & (data['sar_flip'] > 0), 'signal'] = 1
data.loc[(data['regime'] == -1) & (data['sar_flip'] < 0), 'signal'] = -1
position = pd.Series(np.nan, index=data.index)
position.loc[data['signal'] != 0] = data['signal']
position.loc[(data['signal'] == 0) & (data['sar_flip'] != 0)] = 0
data['position'] = position.ffill().fillna(0)

In [55]:
initial_capital = 100000.0
data['market_return'] = data['close'].pct_change()
data['strategy_return'] = data['position'].shift(1) * data['market_return']
data['equity_curve'] = (1 + data['strategy_return']).cumprod() * initial_capital
data['buy_hold_curve'] = (1 + data['market_return']).cumprod() * initial_capital

In [56]:
data.fillna(0, inplace=True)
total_return_strategy = (data['equity_curve'].iloc[-1] / initial_capital) - 1
total_return_bh = (data['buy_hold_curve'].iloc[-1] / initial_capital) - 1
sharpe_ratio = (data['strategy_return'].mean() / (data['strategy_return'].std() + 1e-9)) * np.sqrt(252)
peak = data['equity_curve'].cummax()
drawdown = (data['equity_curve'] - peak) / peak
max_drawdown = drawdown.min()

print(f"--- Performance Results ---")
print(f"Strategy Final Portfolio Value: ${data['equity_curve'].iloc[-1]:,.2f}")
print(f"Buy & Hold Final Portfolio Value: ${data['buy_hold_curve'].iloc[-1]:,.2f}")
print(f"Strategy Total Return: {total_return_strategy:.2%}")
print(f"Buy & Hold Total Return: {total_return_bh:.2%}")
print(f"Strategy Sharpe Ratio: {sharpe_ratio:.2f}")
print(f"Strategy Maximum Drawdown: {max_drawdown:.2%}")

--- Performance Results ---
Strategy Final Portfolio Value: $148,667.90
Buy & Hold Final Portfolio Value: $277,218.60
Strategy Total Return: 48.67%
Buy & Hold Total Return: 177.22%
Strategy Sharpe Ratio: 0.40
Strategy Maximum Drawdown: -24.52%


In [58]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1,
                    subplot_titles=('SPY Price with Trading Signals', 'Strategy Performance vs. Buy & Hold'))

fig.add_trace(go.Scatter(x=data.index, y=data['close'], name='SPY Close', line=dict(color='black')), row=1, col=1)
fig.add_trace(go.Scatter(x=data.index, y=data['mcginley'], name='McGinley Dynamic', line=dict(color='blue', dash='dash')), row=1, col=1)
fig.add_trace(go.Scatter(x=data.index, y=data['parabolic_sar'], name='Parabolic SAR', mode='markers', marker=dict(color='grey', size=3)), row=1, col=1)
buy_signals = data[data['signal'] == 1]
fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['close'], name='Buy Signal', mode='markers',
                         marker=dict(color='green', symbol='triangle-up', size=10)), row=1, col=1)
sell_signals = data[data['signal'] == -1]
fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['close'], name='Short Signal', mode='markers',
                          marker=dict(color='red', symbol='triangle-down', size=10)), row=1, col=1)
fig.add_trace(go.Scatter(x=data.index, y=data['equity_curve'], name='Strategy Equity Curve', line=dict(color='blue')), row=2, col=1)
fig.add_trace(go.Scatter(x=data.index, y=data['buy_hold_curve'], name='Buy & Hold Curve', line=dict(color='orange', dash='dash')), row=2, col=1)
fig.update_layout(height=800, title_text='McGinley Dynamic + Parabolic SAR Strategy Analysis', template='plotly_white')

fig.show()